## Fine-Tune InceptionV3 on CIFAR10

- CIFAR10 is a dataset with **10 classes**
- InceptionV3 is a pre-trained model by ImageNet with **1000 classes**

In order to output reliable prediction sets, we need to fine-tune the full-connected layer of model

# 1. Load Model

Generally, a model has two mode: train mode and evaluation mode. 

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

# check GPU status
print("Is CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

# load pre-trained model InceptionV3 and set mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.inception_v3(pretrained=True)

Is CUDA available: True
Device count: 1
Device name: NVIDIA GeForce RTX 3060 Ti


# 1. Fine-Tune prepare -- load train data set

Load a CIFAR10 as the train data to fine-tune Inception

In [2]:
import torchvision.transforms as transforms               # include image preprocess tools
from torchvision.datasets import CIFAR10, CIFAR100        # for loading images from Pytorch CIFAR
from torch.utils.data import DataLoader
import os

# reprocess the images from CIFAR
data_transform = transforms.Compose([ 
    transforms.Resize((299, 299)),
    transforms.ToTensor(),          # transfer to tensor
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # normalize
])

# make sure CIFAR10 and CIFAR100 in the following adress:  THE_ADRESS_IN_OUTPUT/data
print("Current Working Directory:", os.getcwd())

# load data set for training
pre_train_dataset = CIFAR10(root="./data", train=True, download=True,transform=data_transform)
pre_test_dataset = CIFAR10(root="./data", train=True, download=True, transform=data_transform)

pre_train_loader = DataLoader(pre_train_dataset, batch_size=32, shuffle=True)
pre_test_loader = DataLoader(pre_test_dataset, batch_size=32, shuffle=False)

Current Working Directory: C:\Users\jiayang\ipynb
Files already downloaded and verified
Files already downloaded and verified


# 2.1 Fine-Tune

- adjust output dimension from 1000 to 10
- Freeze parameters in convolution layers and unlock parameters in fc layers
- Train fc layer with CIFAR10

In [3]:
import torch.optim as optim # optimizer

# adjust output dimension 1000 --> 10
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

# Freeze parameters in convolution layers
for param in model.parameters():
    param.requires_grad = False

# unlock parameters in fc layers
for param in model.fc.parameters():
    param.requires_grad = True
    
# Train fc layer
loss_function = nn.CrossEntropyLoss()   # define loss function
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)  # define optimize for fc layer

epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0 # total loss in this epoch
    for images, labels in pre_train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # front propagation
        outputs = model(images)
        loss = loss_function(outputs.logits, labels)

        # back propagation
        optimizer.zero_grad()  # clear gradient
        loss.backward()        # optimize parameters by back propagation
        optimizer.step()       # update the parameters

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}")

# Test model after training
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in pre_test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        #  calculate the correct rate after training
        _, predicted = torch.max(outputs, 1)  # outputs: [batch_size, num_classes]  --torch.max--> max_predicted_prob, max_predicted_prob_label
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Epoch [1/5], Loss: 1627.7409
Epoch [2/5], Loss: 1437.9260
Epoch [3/5], Loss: 1433.9973
Epoch [4/5], Loss: 1434.9058
Epoch [5/5], Loss: 1431.0698
Test Accuracy: 77.55%


# 2.2 Check Fine-Tune Result

We load a batch of test data to make a quick-test, which check the output dimension of the model after fine-tuning. The expected output is [32, 10].

In [4]:
images, labels = next(iter(pre_test_loader))
images = images.to(device)

outputs = model(images)
print("Model Output Shape:", outputs.shape)  # [batch_size, 10]

Model Output Shape: torch.Size([32, 10])


# 1.5 Save fine-tuned Model

Save current parameters and load the trained InceptionV3 in future by following steps:
- model = models.inception_v3(pretrained=False)  # Noticed: pretrained=False!!!
- model.fc = nn.Linear(model.fc.in_features, 10)
- model_path = "C:\Users\jiayang\ipynb\trainedModel\Inception_V3_CIFAR10.pth"
- model.load_state_dict(torch.load(model_path))
- model.eval()
- print(f"Model loaded from {model_path}")

In [5]:
images, labels = next(iter(pre_test_loader))
images = images.to(device)

outputs = model(images)
print("Model Output Shape:", outputs.shape)  # [batch_size, 10]

# create directory
save_dir = "C:\\Users\\jiayang\\ipynb\\trainedModel" # your save save path
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# save model data
model_path = os.path.join(save_dir, "Inception_V3_CIFAR10.pth")
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model Output Shape: torch.Size([32, 10])
Model saved to C:\Users\jiayang\ipynb\trainedModel\Inception_V3_CIFAR10.pth


In [9]:
import numpy as np
 
model = model.to(device)
model.eval()
print(f"Model loaded successfully from {model_path}")

dataset = CIFAR10(root="./data", train=False, download=True, transform=data_transform)

# randomly select 25 pictures
subset_indices = np.random.choice(len(dataset), 25, replace=False)
subset = torch.utils.data.Subset(dataset, subset_indices)
loader = torch.utils.data.DataLoader(subset, batch_size=1, shuffle=False)

# CIFAR-10 labels
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

print("\nPredictions on 25 images:")
for i, (image, true_label) in enumerate(loader):
    image, true_label = image.to(device), true_label.item()
    
    outputs = model(image)
    softmax_probs = torch.softmax(outputs, dim=1).squeeze(0) 
    
    print(f"Image {i+1}:")
    print(f"  Correct Label Index: {true_label} ({classes[true_label]})")
    print(f"  Softmax Probabilities: {softmax_probs.tolist()}") 
    print()

Model loaded successfully from C:\Users\jiayang\ipynb\trainedModel\Inception_V3_CIFAR10.pth
Files already downloaded and verified

Predictions on 25 images:
Image 1:
  Correct Label Index: 3 (cat)
  Softmax Probabilities: [0.1322980523109436, 0.11223164945840836, 0.12616108357906342, 0.38432055711746216, 0.017622916027903557, 0.0385405495762825, 0.14352673292160034, 0.022274896502494812, 0.0028038471937179565, 0.02021963521838188]

Image 2:
  Correct Label Index: 1 (automobile)
  Softmax Probabilities: [0.08567936718463898, 0.09526278078556061, 0.45295819640159607, 0.10355601459741592, 0.057373542338609695, 0.08024773001670837, 0.08847623318433762, 0.013160191476345062, 0.014823965728282928, 0.008462033234536648]

Image 3:
  Correct Label Index: 6 (frog)
  Softmax Probabilities: [0.11253733932971954, 0.0036756275221705437, 0.5619462132453918, 0.014866849407553673, 0.04043850302696228, 0.0012644536327570677, 0.23583929240703583, 0.01310797967016697, 0.015785101801156998, 0.0005386619595